In [6]:
import os
import cv2
import pandas as pd

In [2]:
data_dir = os.path.join(os.getcwd(), 'data')
data_dir

'/home/sangjun/Desktop/sound/data'

In [20]:
import os
import pandas as pd

# 파일이 저장되어 있는 경로 설정 (사용자가 제공한 경로)
data_dir = '/home/sangjun/Desktop/sound/data'

# data_dir에서 .txt 파일만을 추출
file_paths = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.txt')]

# 각 파일을 읽어서 타격 시점, 위치, 도구 정보를 저장할 리스트
all_data = []

# 파일을 읽어 타격 시점, 위치, 도구 정보를 추출
for file_path in file_paths:
    file_name = os.path.basename(file_path)  # 파일 이름 추출 (예: 1.txt)
    file_number = file_name.split('.')[0]    # 파일 번호 추출 (예: 1)
    
    with open(file_path, 'r') as file:
        for line in file:
            # 탭으로 구분된 데이터를 분리
            parts = line.strip().split("\t")
            if len(parts) > 4:
                hit_time = parts[0]  # 타격 시점
                tool_info = parts[4]  # 타격 위치와 도구
                
                # 1차적으로 _로 분리한 후 공백으로 다시 분리
                if "_" in tool_info:
                    tool_parts = tool_info.split('_')
                else:
                    tool_parts = tool_info.split()
                
                # 타격 위치와 도구를 분리
                if len(tool_parts) == 2:
                    hit_location = tool_parts[0]  # 타격 위치
                    hit_tool = tool_parts[1]  # 타격 도구
                else:
                    hit_location = tool_info
                    hit_tool = ''  # 도구 정보가 없을 경우
                
                # 각 데이터에 파일 번호 추가
                all_data.append([hit_time, hit_location, hit_tool, file_number])

# 데이터를 DataFrame으로 정리
df = pd.DataFrame(all_data, columns=["Hit Time", "Hit Location", "Hit Tool", "File Number"])

# 데이터프레임을 출력
print(df)


         Hit Time Hit Location   Hit Tool File Number
0    00:00:00.948        steer     driver           3
1    00:00:03.420        steer     driver           3
2    00:00:05.860        steer     driver           3
3    00:00:08.130        steer     driver           3
4    00:00:10.160        steer     driver           3
..            ...          ...        ...         ...
430  00:10:51.847         seat  drumstick           2
431  00:10:54.683         seat  drumstick           2
432  00:10:57.164         seat  drumstick           2
433  00:11:00.012         seat  drumstick           2
434  00:11:09.253         seat  drumstick           2

[435 rows x 4 columns]


In [24]:
df_grouped

,Hit Location,Hit Tool,Count
0,dashboard,driver,51
1,dashboard,drumstick,64
2,dashboard,rubber,34
3,seat,driver,49
4,seat,drumstick,57
5,seat,rubber,37
6,steer,driver,44
7,steer,drumstick,62
8,steer,rubber,37


In [25]:
# 각 위치에 각 도구로 타격한 경우의 수를 계산
df_grouped = df.groupby(['Hit Location', 'Hit Tool']).size().reset_index(name='Count')
df_grouped

,Hit Location,Hit Tool,Count
0,dashboard,driver,51
1,dashboard,drumstick,64
2,dashboard,rubber,34
3,seat,driver,49
4,seat,drumstick,57
5,seat,rubber,37
6,steer,driver,44
7,steer,drumstick,62
8,steer,rubber,37


In [16]:
file_paths

['/home/sangjun/Desktop/sound/data/1.txt',
 '/home/sangjun/Desktop/sound/data/2.txt',
 '/home/sangjun/Desktop/sound/data/3.txt']

In [27]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 68.1 MB/s eta 0:00:00:00:0100:01


In [38]:
import os
import pandas as pd
import numpy as np
from scipy.io import wavfile

# 시간 문자열을 초 단위로 변환하는 함수 (예: '00:00:00.948' -> 0.948초)
def time_to_seconds(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + float(s)

# 타격 시점에 따른 소리의 강/약을 구분하는 함수
def calculate_rms(audio_signal):
    return np.sqrt(np.mean(audio_signal**2))

# 각 타격 위치와 도구별로 threshold 설정
thresholds = {
    ('dashboard', 'rubber'): 0.015,
    ('dashboard', 'driver'): 0.020,
    ('dashboard', 'drumstick'): 0.025,
    ('seat', 'rubber'): 0.017,
    ('seat', 'driver'): 0.022,
    ('seat', 'drumstick'): 0.030,
    ('steer', 'rubber'): 0.1,
    ('steer', 'driver'): 0.021,
    ('steer', 'drumstick'): 0.028,
}

# 타격 소리에서 타격 시점의 구간을 추출하고, RMS 에너지를 계산하여 강/약을 구분하는 함수
def classify_impact_strength(wav_file, hit_times, sample_rate, tool_info):
    strengths = []
    for i, hit_time in enumerate(hit_times):
        # 타격 시점의 초 단위 시간을 샘플 인덱스로 변환
        hit_seconds = time_to_seconds(hit_time)  # 시간을 초로 변환
        hit_sample_index = int(hit_seconds * sample_rate)
        
        # 타격 소리의 구간을 설정 (예: 타격 전후 0.1초 구간)
        start_sample = max(0, hit_sample_index - int(0.05 * sample_rate))  # 타격 전 0.05초
        end_sample = min(len(wav_file), hit_sample_index + int(0.05 * sample_rate))  # 타격 후 0.05초
        
        # 해당 구간의 RMS 에너지 계산
        rms_energy = calculate_rms(wav_file[start_sample:end_sample])
        
        # 타격 위치와 도구에 따른 threshold 설정
        location, tool = tool_info[i]
        threshold = thresholds.get((location, tool), 0.02)  # 기본값은 0.02
        print('hit_time__loc__tool__rms_energy',hit_time,location,tool,rms_energy)

        # 강/약 기준에 따라 분류
        if rms_energy > threshold:
            strengths.append("strong")
        else:
            strengths.append("weak")
    
    return strengths


# .txt 파일들을 읽고 처리
file_paths = sorted([os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.txt')])

all_data = []

# 각 파일을 읽어서 타격 시점, 위치, 도구 정보를 저장한 후 소리 파일 분석
for file_path in file_paths:
    file_name = os.path.basename(file_path)  # 파일 이름 추출 (예: 1.txt)
    file_number = file_name.split('.')[0]    # 파일 번호 추출 (예: 1)
    
    # wav 파일과 대응되는 소리 파일 로드
    wav_file_path = os.path.join(data_dir, f"{file_number}.wav")
    
    if os.path.exists(wav_file_path):
        # wav 파일 읽기
        sample_rate, audio_data = wavfile.read(wav_file_path)
        
        with open(file_path, 'r') as file:
            hit_times = []
            tool_info = []
            
            for line in file:
                # 탭으로 구분된 데이터를 분리
                parts = line.strip().split("\t")
                if len(parts) > 4:
                    hit_time = parts[0]  # 타격 시점
                    tool_info_text = parts[4]  # 타격 위치와 도구
                    
                    # 타격 시간 저장
                    hit_times.append(hit_time)
                    
                    # 타격 위치와 도구 구분
                    tool_parts = tool_info_text.split('_') if "_" in tool_info_text else tool_info_text.split()
                    if len(tool_parts) == 2:
                        hit_location = tool_parts[0]
                        hit_tool = tool_parts[1]
                    else:
                        hit_location = tool_info_text
                        hit_tool = ''
                    
                    tool_info.append((hit_location, hit_tool))
            
            # 타격 시점에 따른 강/약 정보 추출
            impact_strengths = classify_impact_strength(audio_data, hit_times, sample_rate, tool_info)
            
            # 데이터를 저장 (타격 시점, 위치, 도구, 강/약)
            for i, hit_time in enumerate(hit_times):
                all_data.append([hit_time, tool_info[i][0], tool_info[i][1], impact_strengths[i], file_number])

# 결과를 DataFrame으로 정리
df = pd.DataFrame(all_data, columns=["Hit Time", "Hit Location", "Hit Tool", "Impact Strength", "File Number"])

# 데이터프레임 출력
print(df)

csv_output_path = os.path.join(data_dir, 'data_summary.csv')
df.to_csv(csv_output_path, index=False)



/tmp/ipykernel_162664/4221209866.py:72: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio_data = wavfile.read(wav_file_path)
/tmp/ipykernel_162664/4221209866.py:13: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.mean(audio_signal**2))


hit_time__loc__tool__rms_energy 00:00:02.180 steer rubber 6161.127449904624
hit_time__loc__tool__rms_energy 00:00:06.230 steer rubber 5853.597058506505
hit_time__loc__tool__rms_energy 00:00:10.500 steer rubber 3318.549008390329
hit_time__loc__tool__rms_energy 00:00:15.780 steer rubber 5354.96691030214
hit_time__loc__tool__rms_energy 00:00:30.515 steer rubber 817.707483915673
hit_time__loc__tool__rms_energy 00:00:43.666 steer rubber 3551.0273351093592
hit_time__loc__tool__rms_energy 00:00:47.969 steer rubber 3295.0701548176585
hit_time__loc__tool__rms_energy 00:00:52.878 steer rubber nan
hit_time__loc__tool__rms_energy 00:01:16.394 steer rubber nan
hit_time__loc__tool__rms_energy 00:01:19.697 steer rubber nan
hit_time__loc__tool__rms_energy 00:01:24.060 steer rubber nan
hit_time__loc__tool__rms_energy 00:01:27.606 steer rubber 5364.039475590351
hit_time__loc__tool__rms_energy 00:01:39.969 steer rubber 5015.266207216891
hit_time__loc__tool__rms_energy 00:01:43.091 steer rubber 6716.95127

In [41]:
import os
import pandas as pd
import numpy as np
from scipy.io import wavfile

# 시간 문자열을 초 단위로 변환하는 함수 (예: '00:00:00.948' -> 0.948초)
def time_to_seconds(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + float(s)

# 소리 신호에서 최대 진폭을 계산하는 함수
def calculate_max_amplitude(audio_signal):
    return np.max(np.abs(audio_signal))

# 각 타격 위치와 도구별로 threshold 설정
thresholds = {
    ('dashboard', 'rubber'): 0.015,
    ('dashboard', 'driver'): 0.020,
    ('dashboard', 'drumstick'): 0.025,
    ('seat', 'rubber'): 0.017,
    ('seat', 'driver'): 0.022,
    ('seat', 'drumstick'): 0.030,
    ('steer', 'rubber'): 0.1,
    ('steer', 'driver'): 0.021,
    ('steer', 'drumstick'): 0.028,
}

# 타격 소리에서 타격 시점의 후 0.5초 구간을 추출하고, 최대 진폭을 계산하여 강/약을 구분하는 함수
def classify_impact_strength(wav_file, hit_times, sample_rate, tool_info):
    strengths = []
    for i, hit_time in enumerate(hit_times):
        # 타격 시점의 초 단위 시간을 샘플 인덱스로 변환
        hit_seconds = time_to_seconds(hit_time)  # 시간을 초로 변환
        hit_sample_index = int(hit_seconds * sample_rate)
        
        # 타격 시점 후 0.5초 구간 설정
        start_sample = hit_sample_index  # 타격 시점
        end_sample = min(len(wav_file), hit_sample_index + int(0.5 * sample_rate))  # 타격 후 0.5초
        
        # 해당 구간의 최대 진폭 계산
        max_amplitude = calculate_max_amplitude(wav_file[start_sample:end_sample])
        
        # 타격 위치와 도구에 따른 threshold 설정
        location, tool = tool_info[i]
        threshold = thresholds.get((location, tool), 0.02)  # 기본값은 0.02
        print('hit_time__loc__tool__max_amplitude', hit_time, location, tool, max_amplitude)

        # 강/약 기준에 따라 분류
        if max_amplitude > threshold:
            strengths.append("strong")
        else:
            strengths.append("weak")
    
    return strengths

# .txt 파일들을 읽고 처리
file_paths = sorted([os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.txt')])

all_data = []

# 각 파일을 읽어서 타격 시점, 위치, 도구 정보를 저장한 후 소리 파일 분석
for file_path in file_paths:
    file_name = os.path.basename(file_path)  # 파일 이름 추출 (예: 1.txt)
    file_number = file_name.split('.')[0]    # 파일 번호 추출 (예: 1)
    
    # wav 파일과 대응되는 소리 파일 로드
    wav_file_path = os.path.join(data_dir, f"{file_number}.wav")
    
    if os.path.exists(wav_file_path):
        # wav 파일 읽기
        sample_rate, audio_data = wavfile.read(wav_file_path)
        
        with open(file_path, 'r') as file:
            hit_times = []
            tool_info = []
            
            for line in file:
                # 탭으로 구분된 데이터를 분리
                parts = line.strip().split("\t")
                if len(parts) > 4:
                    hit_time = parts[0]  # 타격 시점
                    tool_info_text = parts[4]  # 타격 위치와 도구
                    
                    # 타격 시간 저장
                    hit_times.append(hit_time)
                    
                    # 타격 위치와 도구 구분
                    tool_parts = tool_info_text.split('_') if "_" in tool_info_text else tool_info_text.split()
                    if len(tool_parts) == 2:
                        hit_location = tool_parts[0]
                        hit_tool = tool_parts[1]
                    else:
                        hit_location = tool_info_text
                        hit_tool = ''
                    
                    tool_info.append((hit_location, hit_tool))
            
            # 타격 시점에 따른 강/약 정보 추출
            impact_strengths = classify_impact_strength(audio_data, hit_times, sample_rate, tool_info)
            
            # 데이터를 저장 (타격 시점, 위치, 도구, 강/약)
            for i, hit_time in enumerate(hit_times):
                all_data.append([hit_time, tool_info[i][0], tool_info[i][1], impact_strengths[i], file_number])

# 결과를 DataFrame으로 정리
df = pd.DataFrame(all_data, columns=["Hit Time", "Hit Location", "Hit Tool", "Impact Strength", "File Number"])

# 데이터프레임 출력
print(df)

# CSV 파일로 저장
csv_output_path = os.path.join(data_dir, 'data_summary.csv')
df.to_csv(csv_output_path, index=False)


/tmp/ipykernel_162664/3089190728.py:71: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio_data = wavfile.read(wav_file_path)


hit_time__loc__tool__max_amplitude 00:00:02.180 steer rubber 180457472
hit_time__loc__tool__max_amplitude 00:00:06.230 steer rubber 169852928
hit_time__loc__tool__max_amplitude 00:00:10.500 steer rubber 148796672
hit_time__loc__tool__max_amplitude 00:00:15.780 steer rubber 249647104
hit_time__loc__tool__max_amplitude 00:00:30.515 steer rubber 157948928
hit_time__loc__tool__max_amplitude 00:00:43.666 steer rubber 190057216
hit_time__loc__tool__max_amplitude 00:00:47.969 steer rubber 186706432
hit_time__loc__tool__max_amplitude 00:00:52.878 steer rubber 202745088
hit_time__loc__tool__max_amplitude 00:01:16.394 steer rubber 109523200
hit_time__loc__tool__max_amplitude 00:01:19.697 steer rubber 128027392
hit_time__loc__tool__max_amplitude 00:01:24.060 steer rubber 96210944
hit_time__loc__tool__max_amplitude 00:01:27.606 steer rubber 228608768
hit_time__loc__tool__max_amplitude 00:01:39.969 steer rubber 49615872
hit_time__loc__tool__max_amplitude 00:01:43.091 steer rubber 659641856
hit_time

In [64]:
import cv2
import numpy as np

# 타격 시점에서 전후로 설정된 시간 동안의 프레임 추출
def extract_stacked_images(video_path, hit_time, frame_rate=30, pre_interval=0.3, post_interval=0.5):
    cap = cv2.VideoCapture(video_path)
    hit_time_seconds = time_to_seconds(hit_time)  # 타격 시점을 초로 변환
    
    # 타격 전후 프레임 범위 계산
    pre_start_frame = int((hit_time_seconds - pre_interval) * frame_rate)
    post_end_frame = int((hit_time_seconds + post_interval) * frame_rate)
    
    pre_frame_count = int(pre_interval * frame_rate)  # 타격 전 프레임 개수
    post_frame_count = int(post_interval * frame_rate)  # 타격 후 프레임 개수

    pre_frames = []
    post_frames = []

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 타격 전 프레임 9개 추출
        if pre_start_frame <= frame_count < pre_start_frame + pre_frame_count:
            pre_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))  
        # 타격 후 프레임 15개 추출
        elif post_end_frame - post_frame_count <= frame_count < post_end_frame:
            post_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)) 
        
        frame_count += 1
        if frame_count >= post_end_frame:
            break

    cap.release()
    
    if len(pre_frames) < 3 or len(post_frames) < 3:
        raise ValueError("Not enough frames for stacking.")
    
    # 타격 전 프레임에서 처음, 중간, 끝 프레임을 스택
    pre_stack = np.stack((pre_frames[0], pre_frames[len(pre_frames)//2], pre_frames[-1]), axis=2)
    # 타격 후 프레임에서 처음, 중간, 끝 프레임을 스택
    post_stack = np.stack((post_frames[0], post_frames[len(post_frames)//2], post_frames[-1]), axis=2)

    return pre_stack, post_stack

# 시간 문자열을 초로 변환하는 함수 (예: '00:00:00.948' -> 0.948초)
def time_to_seconds(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + float(s)

# 예시 사용
video_path = "./data/VR_1.mp4"
# hit_time = "00:00:10.500"  # 타격 시점 예시
hit_time = "00:00:10.500"  # 타격 시점 예시
pre_stack_img, post_stack_img = extract_stacked_images(video_path, hit_time)

# 결과 확인을 위해 이미지 저장 (옵션)
cv2.imwrite("pre_stack_img.jpg", pre_stack_img)
cv2.imwrite("post_stack_img.jpg", post_stack_img)


True

In [67]:
import cv2
import numpy as np

# 타격 시점에서 전후로 설정된 시간 동안의 프레임 추출
def extract_stacked_images(video_path, hit_time, frame_rate=30, pre_interval=0.5, post_interval=0.5):
    cap = cv2.VideoCapture(video_path)
    hit_time_seconds = time_to_seconds(hit_time)  # 타격 시점을 초로 변환
    
    # 타격 전후 프레임 범위 계산
    pre_start_frame = int((hit_time_seconds - pre_interval) * frame_rate)
    post_end_frame = int((hit_time_seconds + post_interval) * frame_rate)
    
    pre_frame_count = int(pre_interval * frame_rate)  # 타격 전 프레임 개수
    post_frame_count = int(post_interval * frame_rate)  # 타격 후 프레임 개수

    pre_frames = []
    post_frames = []

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 타격 전 프레임 9개 추출
        if pre_start_frame <= frame_count < pre_start_frame + pre_frame_count:
            pre_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))  
        # 타격 후 프레임 15개 추출
        elif post_end_frame - post_frame_count <= frame_count < post_end_frame:
            post_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)) 
        
        frame_count += 1
        if frame_count >= post_end_frame:
            break

    cap.release()
    
    if len(pre_frames) < 3 or len(post_frames) < 3:
        raise ValueError("Not enough frames for stacking.")
    
    # 타격 전 프레임에서 처음, 중간, 끝 프레임을 스택
    pre_stack = np.stack((pre_frames[0], pre_frames[len(pre_frames)//2], pre_frames[-1]), axis=2)
    # 타격 후 프레임에서 처음, 중간, 끝 프레임을 스택
    post_stack = np.stack((post_frames[0], post_frames[len(post_frames)//2], post_frames[-1]), axis=2)

    return pre_stack, post_stack

# 시간 문자열을 초로 변환하는 함수 (예: '00:00:00.948' -> 0.948초)
def time_to_seconds(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + float(s)

# 예시 사용
video_path = "./data/VR_1.mp4"
# hit_time = "00:00:10.500"  # 타격 시점 예시
hit_time = "00:01:27.588"  # 타격 시점 예시
pre_stack_img, post_stack_img = extract_stacked_images(video_path, hit_time)

# 결과 확인을 위해 이미지 저장 (옵션)
cv2.imwrite("pre_stack_img2.jpg", pre_stack_img)
cv2.imwrite("post_stack_img2.jpg", post_stack_img)


True